In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("../datasets/transaction_history.csv")

In [3]:
mcc = pd.read_csv("../data/MCC codes.csv")

In [4]:
data = data.merge(mcc , on = 'originalMcc')

In [5]:
data.query('`Business group` == "Продаж в роздріб" ').Category.unique()

array(['Різноманітні продовольчі магазини – ринки, магазини',
       'Бакалійні магазини, супермаркети',
       'Різні товари загального призначення',
       'Магазини, що торгують зі зниженими цінами'], dtype=object)

In [6]:
import time

In [36]:
current_month = current_week = int(time.strftime('%m'))

In [7]:
current_week = int(time.strftime('%w'))

### Hight Level Report 

In [8]:
custom_categories = {
    'MCC_Categories' :  [{'originalMcc' : '5812', 'custom_category': 'Харчування'},{'originalMcc' : '5499', 'custom_category': 'Харчування'},{'originalMcc' : '5411', 'custom_category': 'Харчування'}],
    'Description_Categories' : [{'description' : 'Петро С.', 'custom_category': 'Квартплата'} , {'description': '535129****2010', 'custom_category': 'Комунальний платіж'}]
}

In [9]:
mcc_categories = pd.DataFrame([category for category in custom_categories['MCC_Categories']])
mcc_categories['originalMcc'] = mcc_categories['originalMcc'].astype(int)
data_with_mcc_categories = data.merge(mcc_categories,on='originalMcc', how='left')
description_categories = pd.DataFrame([category for category in custom_categories['Description_Categories']])
data_full = data_with_mcc_categories.merge(description_categories, on='description', how='left')
data_full['custom_category_x']=  data_full['custom_category_x'].fillna(data_full['custom_category_y'])
data_full['custom_category_x']=  data_full['custom_category_x'].fillna(data_full['Business group'])
data_full  = data_full.rename(columns={'custom_category_x':'custom_category'})
del data_full['custom_category_y']

In [48]:
data_full['custom_category'].unique()

array(['Харчування', 'Фінанси', 'Різні магазини', 'Онлайн-майданчики',
       'Квартплата', 'Постачальники послуг', 'Транспортування',
       'Комунальні послуги', 'Комунальний платіж', 'Продаж в роздріб'],
      dtype=object)

In [15]:
data_full.query("week_number == 1").groupby( ['user_name', 'Business group', 'custom_category'] ).amount.sum().to_frame()

amount
user_name Business group    custom_category          
Artem     Онлайн-майданчики Онлайн-майданчики    2713
          Продаж в роздріб  Харчування           4479
          Різні магазини    Різні магазини       2082
                            Харчування            120
          Фінанси           Квартплата           7000
                            Фінанси             11177
Ira       Продаж в роздріб  Харчування             67
          Транспортування   Транспортування       217
          Фінанси           Фінанси              6541

In [23]:
Balances = pd.read_csv("../datasets/balances.csv")

### Weekly balance report

In [57]:
Weekly = data_full.query("week_number == 1").groupby( ['custom_category'] ).amount.sum().reset_index().merge(Balances.query('Period == "w"'),how='right')
Weekly = Weekly.fillna(0)

In [64]:
display(Weekly)
for category in Weekly.custom_category.unique():

    category_df = Weekly.query("custom_category == @category")

    availible = category_df['Balance'] -category_df['amount']
    availible = availible[0]
    if availible > 0 : 
        balance_report_body = f"For category {category}. You have {availible}. UAH"

    else :
        balance_report_body = f"For categpry {category}. You exceeded the weekly limit."

balance_report_body

,custom_category,amount,Balance,Period
0,Харчування,4666,3000,w


'For categpry Харчування. You exceeded the weekly limit.'

### Month Balance report 

In [59]:
data_full

,Unnamed: 0,id,time,description,originalMcc,amount,receiptId,type,full_date,month_number,week_number,user_name,telegram_id,Category,Business group,custom_category
0,0,QybTdwmdg1UxQYorUw,1736090319,Glovo,5499,434,E8C4-AH3E-CEA7-H0E8,outcome,2025-01-05,1,1,Artem,533452580,"Різноманітні продовольчі магазини – ринки, маг...",Продаж в роздріб,Харчування
1,1,X2L4UyGbf3mHQnmHCg,1736016367,Анатолій К.,4829,715,T0E9-MC6C-3P2P-A72C,outcome,2025-01-04,1,1,Artem,533452580,Грошові перекази,Фінанси,Фінанси
2,2,dqY1XO0j5LgB88i-DUL2,1735950685,Glovo,5499,291,NaN,outcome,2025-01-04,1,1,Artem,533452580,"Різноманітні продовольчі магазини – ринки, маг...",Продаж в роздріб,Харчування
3,3,6cexPC-Cv8T6zid9SkR7vBBmR7M,1735950685,Скасування. Glovo,5499,288,NaN,income,2025-01-04,1,1,Artem,533452580,"Різноманітні продовольчі магазини – ринки, маг...",Продаж в роздріб,Харчування
4,4,5fRWZpgp0bXROL-_vA,1735920286,Glovo,5499,521,1914-H328-TBHP-KPTP,outcome,2025-01-03,1,1,Artem,533452580,"Різноманітні продовольчі магазини – ринки, маг...",Продаж в роздріб,Харчування
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,41,CbFMNVPPwAs5p5aj5Q,1733589765,SIMI #4133,5499,457,HKAA-P721-A2EC-1641,outcome,2024-12-07 00:00:00,12,49,Ira,820980192,"Різноманітні продовольчі магазини – ринки, маг...",Продаж в роздріб,Харчування
115,42,GI1HNsfLIhmssLStFQ,1733581779,Uklon,4121,100,058K-T735-TE7P-976M,outcome,2024-12-07 00:00:00,12,49,Ira,820980192,Лімузини та таксі,Транспортування,Транспортування
116,43,l9iYR2quEjfbjjqERA,1733501173,Термінал EasyPay,4829,200,NaN,income,2024-12-06 00:00:00,12,49,Ira,820980192,Грошові перекази,Фінанси,Фінанси
117,44,bZmTP9ZZiPV1f_pUJQ,1733488963,473121****4300,4829,200,AP5B-CAK0-T1X3-BHT7,outcome,2024-12-06 00:00:00,12,49,Ira,820980192,Грошові перекази,Фінанси,Фінанси


In [60]:
Monthly = data_full.query("month_number == @current_month").groupby( ['custom_category'] ).amount.sum().reset_index().merge(Balances.query('Period == "m"'),how='right')
Monthly = Monthly.fillna(0)

In [62]:
display(Monthly)
for category in Monthly.custom_category.unique():

    category_df = Monthly.query("custom_category == @category")

    availible = category_df['Balance'] -category_df['amount']
    availible = availible[0]
    if availible > 0 : 
        balance_report_body = f"For category {category}. You have {availible} UAH."

    else :
        balance_report_body = f"For categpry {category}. You exceeded the weekly limit."

balance_report_body

,custom_category,amount,Balance,Period
0,Транспортування,0.0,2000,m


'For category Транспортування. You have 2000.0 UAH.'

In [63]:
data.groupby('Business group').id.nunique()

Business group
Комунальні послуги       8
Онлайн-майданчики        3
Постачальники послуг     1
Продаж в роздріб        49
Різні магазини          12
Транспортування         18
Фінанси                 28
Name: id, dtype: int64